In [1]:
import pyspark
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


24/02/28 16:15:44 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [10]:
!wc -l fhvhv_tripdata_2021-01.csv.gz

508066 fhvhv_tripdata_2021-01.csv.gz


In [11]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv.gz')

In [12]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [7]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [8]:
import os
# resolution to memory warning
os.environ["PYSPARK_SUBMIT_ARGS"] = "--driver-memory 2g"

import pandas as pd


df.write.parquet('fhvhv/2021/01/') results in below error

```bash
24/02/27 02:18:17 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for ## writers
```

[resolution](https://stackoverflow.com/questions/53407442/total-allocation-exceeds-95-00-960-285-889-bytes-of-heap-memory-pyspark-erro) on above error is to increase the memory.

Consequence of above error was also that the RAM is eaten up and not released after writing? As vscode hangs and cannot save new edits.

In [13]:
df_pandas = pd.read_csv('head.csv')

In [14]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

Use above dtypes to create a spark dataframe

In [15]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

Integer - 4 bytes

Long - 8 bytes

In [16]:
from pyspark.sql import types

change LongType to Integer, StructType is from scala, reformat to PySpark StructType
just so it is more readable from the long string in cell#15

In [17]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [18]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [ ]:
df = df.repartition(24) # makes 24 partitions 0-23

In [ ]:
df.write.parquet('fhvhv/2021/01/', mode="overwrite")

In [ ]:
df = spark.read.parquet('fhvhv/2021/01/')

In [ ]:
df.printSchema()

SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df.show()

In [ ]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [ ]:
crazy_stuff('B02884')

In [ ]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [ ]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

In [ ]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')


In [ ]:
!head -n 10 head.csv